In [63]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import ChatGLM 

# 1. 文档加载与分割
loader = TextLoader(r"E:\一些也许有点用的文件\一些大作业\软件体系结构_大模型\myRAG\data\风格文本1.txt", encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=256, chunk_overlap=32)  # 建议增大chunk_size
docs = text_splitter.split_documents(documents)

# 2. 嵌入模型（推荐中文模型）
embedding = HuggingFaceEmbeddings(
    model_name="shibing624/text2vec-base-chinese",  # 中文嵌入模型
    model_kwargs={'device': 'cuda'}  # 可改为'cuda'
)

# 3. Milvus向量数据库配置
vector_db = Milvus.from_documents(
    docs,
    embedding,
    connection_args={"host": "localhost", "port": "19530"},  # Milvus服务地址
    collection_name="rag_software",
    drop_old=False,
    batch_size=500
)

# 4. 初始化ChatGLM
# 方式1：使用本地部署的ChatGLM3（需提前启动HTTP服务）
from langchain_community.llms import ChatGLM
llm = ChatGLM(
    endpoint_url="http://localhost:8000",  # ChatGLM的API地址
    max_token=2048,
    temperature=0.1
)

# 方式2：直接调用HuggingFace模型（需GPU）
# from langchain.llms import HuggingFacePipeline
# llm = HuggingFacePipeline.from_model_id(
#     model_id="THUDM/chatglm3-6b",
#     task="text-generation",
#     device="cuda"
# )

# 5. 构建对话链
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_db.as_retriever(search_kwargs={"k": 3}),  # 返回top3结果
    memory=memory,
    chain_type="stuff"
)

# 6. 测试问答
query = "藜麦的营养价值有哪些？"
result = qa_chain({"question": query})
print(result["answer"])

Created a chunk of size 3758, which is longer than the specified 256
Created a chunk of size 3104, which is longer than the specified 256
Created a chunk of size 3124, which is longer than the specified 256
Created a chunk of size 2838, which is longer than the specified 256
Created a chunk of size 3046, which is longer than the specified 256


RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
name '_private' is not defined

## 文档增量式地存入milvus数据库

In [1]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import ChatGLM  # 或使用自定义ChatGLM封装

data_dir = r"E:\一些也许有点用的文件\一些大作业\软件体系结构_大模型\myRAG\data"
file_names = [f"风格文本{i}.txt" for i in range(1, 5)]

embedding = HuggingFaceEmbeddings(
    model_name="shibing624/text2vec-base-chinese",  # 中文嵌入模型
    model_kwargs={'device': 'cpu'}
)
print("1 step")

for file_name in file_names:
    file_path = os.path.join(data_dir, file_name)
    
    if not os.path.exists(file_path):
        print(f"文件不存在: {file_path}")
        continue
    
    print(f"正在处理: {file_name}")
    
    # 加载并分割文档
    loader = TextLoader(file_path, encoding="utf-8")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=32, separator="\n")
    docs = text_splitter.split_documents(documents)
    print("2 step")
    vector_db = Milvus.from_documents(
        documents = docs,
        embedding=embedding,
        connection_args={'host': '192.168.3.14', 'port': 19530},  # Milvus服务地址
        collection_name="rag_milvus",
        drop_old=False,
    )
    print(f"已入库: {len(docs)}个片段 | 源文件: {file_name}")

ValidationError: 1 validation error for HuggingFaceEmbeddings
base_url
  Extra inputs are not permitted [type=extra_forbidden, input_value='http://192.168.3.3:11434', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

In [57]:
## version 2

import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import ChatGLM  # 或使用自定义ChatGLM封装

data_dir = r"E:\一些也许有点用的文件\一些大作业\软件体系结构_大模型\myRAG\data"
file_names = [f"风格文本{i}.txt" for i in range(1, 5)]

embedding = HuggingFaceEmbeddings(
    #model_name="shibing624/text2vec-base-chinese",  # 中文嵌入模型
    model_name="moka-ai/m3e-base",  # 中文嵌入模型
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True},
    #query_instruction="为文本生成向量表示用于文本检索"
    )
print("1 step")

for file_name in file_names:
    file_path = os.path.join(data_dir, file_name)
    
    if not os.path.exists(file_path):
        print(f"文件不存在: {file_path}")
        continue
    
    print(f"正在处理: {file_name}")
    
    # 加载并分割文档
    loader = TextLoader(file_path, encoding="utf-8")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=32, separator="\n")
    documents = text_splitter.split_documents(documents)
    print("2 step")
    db = Chroma.from_documents(documents, embedding)
    print(f"已入库: {len(documents)}个片段 | 源文件: {file_name}")

RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
name '_private' is not defined

## MilvusClient方式连接数据库

In [15]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Milvus

embeddings = OllamaEmbeddings(
    model="shaw/dmeta-embedding-zh:latest",
    base_url="http://192.168.3.3:11434"
)
file_path = r"E:\一些也许有点用的文件\一些大作业\软件体系结构_大模型\myRAG\data\风格文本1.txt"
loader = TextLoader(file_path, encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=32, separator="\n")
docs = text_splitter.split_documents(documents)

milvus_conn = {"host": "localhost", "port": "19530"}
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)

vector_db = Milvus.from_documents(
    docs,
    embeddings,
    connection_args=milvus_conn
)
query = "鑫茂园是什么？"
docs = vector_db.similarity_search(query)
print(docs[0].page_content)

Created a chunk of size 304, which is longer than the specified 200
Created a chunk of size 254, which is longer than the specified 200
Created a chunk of size 272, which is longer than the specified 200
Created a chunk of size 215, which is longer than the specified 200
Created a chunk of size 247, which is longer than the specified 200
Failed to create new connection using: fc0c34850a8f49c8a3f8d32c5d284ee9


MilvusException: <MilvusException: (code=2, message=Fail connecting to server on localhost:19530, illegal connection params or server unavailable)>

In [45]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from uuid import uuid4


class VectorStore:
    def __init__(self):
        self.embeddings = OllamaEmbeddings(model="llama3.2:3b")
        self.index = faiss.IndexFlatL2(len(self.embeddings.embed_query("hello world")))
        self.vector_store = FAISS(
            embedding_function=self.embeddings,
            index=self.index,
            docstore=InMemoryDocstore(),
            index_to_docstore_id={}
        )

    def add_docs(self, list_of_docs):
        uuids = [str(uuid4()) for _ in range(len(list_of_docs))]
        self.vector_store.add_documents(documents=list_of_docs, ids=uuids)

    def search_docs(self, query, k=5):
        results = self.vector_store.similarity_search(
            query,
            k=k
        )
        return results


index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
vector_store = FAISS(
            embedding_function=embeddings,
            index=index,
            docstore=InMemoryDocstore(),
            index_to_docstore_id={}
        )


ConnectTimeout: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。